In [1]:
import matplotlib.pyplot as plt
import numpy as np
from Board import Board
from Piece import Piece
from Player import Player
from Game import Game
from shutil import copyfile
import time
import csv
import pdb
import json
from datetime import datetime
import os
import torch

In [2]:
identifier = "20241119190000"
s_model = "PTC"
import_string = 'from ' + s_model + ' import ' + s_model + ' as sm' # create self_play model import string
exec(import_string, globals())
black_model = sm("black_model", identifier)
red_model = sm("red_model", identifier)
red_player = Player(model = red_model, color = "Red") # create the red player assigning model and color
black_player = Player(model = black_model, color = "Black") # create the black player assigning model and color
game = Game(red_player = red_player, black_player = black_player, jump_rule = True, number = 0, side = "black")

cpu
Using initialized parameters.


Dice roll or maximize? R


cpu


Start from saved? n
Start from checkpoint? n


Using initialized parameters.


Dice roll or maximize? R


In [3]:
b, b_pieces = black_model.get_input_vector(game.board, "Black", None)
r, r_pieces = black_model.get_input_vector(game.board, "Black", None)

In [5]:
np.size(b_pieces)

409

In [13]:
r

array([[[0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [1, 1, 1, 1],
        [1, 1, 1, 1],
        [1, 1, 1, 1]],

       [[0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0]],

       [[1, 1, 1, 1],
        [1, 1, 1, 1],
        [1, 1, 1, 1],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0]],

       [[0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0]]])

In [17]:
np.size(b_pieces)

409